In [1]:
!pip install sentence_transformers

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [2]:
!python -m spacy download el_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_ca

✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_md')


In [3]:
'''from shutil import copyfile

copyfile(src = "/kaggle/input/trainer-am/stratification.py", dst = "../working/stratification.py")
copyfile(src = "/kaggle/input/trainer-am/kfold.py", dst = "../working/kfold.py")
copyfile(src = "/kaggle/input/trainer-am/trainer.py", dst = "../working/trainer.py")
copyfile(src = "/kaggle/input/trainer-am/visualizations.py", dst = "../working/visualizations.py")'''

'from shutil import copyfile\n\ncopyfile(src = "/kaggle/input/trainer-am/stratification.py", dst = "../working/stratification.py")\ncopyfile(src = "/kaggle/input/trainer-am/kfold.py", dst = "../working/kfold.py")\ncopyfile(src = "/kaggle/input/trainer-am/trainer.py", dst = "../working/trainer.py")\ncopyfile(src = "/kaggle/input/trainer-am/visualizations.py", dst = "../working/visualizations.py")'

In [4]:
from trainer import Trainer

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [6]:
import matplotlib.pyplot as plt

# Setup

In [7]:
path = "data/iliad_sentiments.csv"
target = "class"

In [8]:
scenarios = {
    "TF-IDF sem lematização": {"lemmatize": False, "sbert": False},
#    "TF-IDF com lematização": {"lemmatize": True, "sbert": False},
#    "SBERT sem lematização": {"lemmatize": False, "sbert": True},
           }

classifiers = {
    "Decision Tree": DecisionTreeClassifier(), 
    "KNN": KNeighborsClassifier(),
    "Random Forest": RandomForestClassifier()
}

STANDARD = "Sem modificação"
WITHOUT_NARRATOR = "Sem narrador"
MERGE_NARRATOR_NEUTRAL = "Junção de narrador e neutro"

data_operations = [STANDARD, WITHOUT_NARRATOR, MERGE_NARRATOR_NEUTRAL]

nro_folds = 5

# Trainer

In [9]:
metrics = {}

for classifier in classifiers:  
    metrics[classifier] = {}
        
    for scenario in scenarios: 
        metrics[classifier][scenario] = {}

for classifier in classifiers:       
    for scenario in scenarios: 
        for operation in data_operations:
            trainer = Trainer(path, target)

            if operation == STANDARD:
                trainer.prepare_data_to_train(lemmatize=scenarios[scenario]["lemmatize"], sbert=scenarios[scenario]["sbert"])
            if operation == WITHOUT_NARRATOR:
                trainer.remove_class(["narrator"])
                trainer.prepare_data_to_train(lemmatize=scenarios[scenario]["lemmatize"], sbert=scenarios[scenario]["sbert"])
            elif operation == MERGE_NARRATOR_NEUTRAL:
                trainer.merge_class(["narrator"], "neutral")
                trainer.prepare_data_to_train(lemmatize=scenarios[scenario]["lemmatize"], sbert=scenarios[scenario]["sbert"])
                
            metrics[classifier][scenario][operation] = trainer.cross_validation(model=classifiers[classifier], nro_folds=nro_folds)
            print(f"{classifier} : {scenario} : {operation}")
            print(metrics[classifier][scenario][operation])
            print("=======================")


Decision Tree : TF-IDF sem lematização : Sem modificação
{'accuracy': [0.3387096774193548, 0.3821138211382114, 0.38524590163934425, 0.4016393442622951, 0.3416666666666667], 'recall': [0.24227738376674546, 0.26065167937909806, 0.2508750341810227, 0.27584768936286574, 0.2443015352162194], 'precision': [0.23072377501725327, 0.2429818922104361, 0.23596601499848283, 0.26153360528360525, 0.23763736263736263], 'especificity': [0.7453622066007387, 0.7575710108604845, 0.7605287217434078, 0.7614376376740306, 0.7302862916422477], 'f1': [0.23635947416143344, 0.2515068170037665, 0.24319223770498827, 0.26850000707852534, 0.24092337338032796]}
Decision Tree : TF-IDF sem lematização : Sem narrador
{'accuracy': [0.25862068965517243, 0.23478260869565218, 0.21929824561403508, 0.13157894736842105, 0.18584070796460178], 'recall': [0.21728395061728392, 0.2176100628930818, 0.17316561844863732, 0.12620545073375264, 0.1495956873315364], 'precision': [0.16512667660208644, 0.15759225650332728, 0.1511042629686697

IndexError: index 4976 is out of bounds for axis 0 with size 488

# Visualizations 

In [ ]:
from visualizations import compare_boxplot

In [ ]:
compare_boxplot(metrics, data_operations, width=14, height=10)

In [ ]:
metrics

In [ ]:
data_operations